In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pulp as plp
import networkx as nx
from ALB_instance_tools import *
from report_functions import *
from milp_models import *
from scenario_trees import *
from collections import namedtuple
import glob
import os
import string
import time
import shutil
from datetime import datetime
from timeit import default_timer as timer



In [ ]:

# NO_TAKTS = 4
# scenario_tree_graph, final_sequences = make_scenario_tree(NO_TAKTS, {'A':0.60, 'B':0.4})
# restricted_graph, restricted_sequences = make_consecutive_luxury_models_restricted_scenario_tree(NO_TAKTS, {'A':0.75, 'B':0.25}, max_consecutive=3, luxury_models=['A', 'B'])
# restricted_total = sum_prob(restricted_sequences)
# restricted_total

In [ ]:
# instance_dicts = [
#    {'fp': "SALBP_benchmark/debugging_ds/instance_n=5_1.alb",'name':'A', 'probability':0.75},
#    {'fp': "SALBP_benchmark/debugging_ds/instance_n=5_2.alb",'name':'B', 'probability':0.25}
# ]


#instance_list = read_instance_folder("SALBP_benchmark/small data set_n=20/")[1:3]
# instance_dicts = [
#    {'fp': "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",'name':'A', 'probability':0.75},
#    {'fp': "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",'name':'B', 'probability':0.25}
# ]
instance_list = [ "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",
   "SALBP_benchmark/small data set_n=20/instance_n=20_3.alb",
   ]
# instance_list = [ "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",
#    "SALBP_benchmark/small data set_n=20/instance_n=20_18.alb",]
#test_instances = create_instance_pair_stochastic(instance_list)


# NO_EQUIPMENT = 4
# seed = 1
# NO_WORKERS =4
# NO_STATIONS = 4
# WORKER_COST = 500
# RECOURSE_COST = WORKER_COST * 2
# TAKT_TIME = 1000
# SEQUENCE_LENGTH = 4
# #MODEL_MIXTURES = {'A':0.34, 'B':0.33, 'C':0.33}
# MODEL_MIXTURES = {'A':0.60, 'B':0.40}



    
# def multi_run(instance_list, milp_model, NO_EQUIPMENT,  NO_WORKERS, 
#               NO_STATIONS, WORKER_COST, TAKT_TIME, SEQUENCE_LENGTH, MODEL_MIXTURES, seed, RECOURSE_COST = WORKER_COST * 2,
#               scenario_generator= make_scenario_tree, file_name = 'test', **kwargs):
#    instance_groups = pair_instances(instance_list, MODEL_MIXTURES)
#    test_results = []
#    group_counter = 0
#    test_instances = []
#    instance_results = []
#    for group in instance_groups:
#       test_instance = MultiModelInstance(model_dicts=group, takt_time=TAKT_TIME, sequence_length=SEQUENCE_LENGTH, max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST, recourse_cost=RECOURSE_COST)
#       print('Running instance', test_instance.name)
#       test_instances.append(test_instance)
#       #create equipment
#       equipment = Equipment( no_tasks= test_instance.no_tasks, no_equipment=NO_EQUIPMENT,no_stations=NO_STATIONS,generation_method='new_method', seed = seed)
#       #create scenario tree
#       print('generating scenario tree')
#       scenario_tree_graph, final_sequences = scenario_generator(SEQUENCE_LENGTH, MODEL_MIXTURES, **kwargs)
#       print('defining problem')
#       milp_prob = milp_model(problem_instance = test_instance, equipment_instance = equipment, sequence_length=SEQUENCE_LENGTH, prod_sequences=final_sequences, **kwargs)
#       #prob = milp_model(test_instance, equipment,test_instance.no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, SEQUENCE_LENGTH, final_sequences, worker_cost=WORKER_COST)
#       solver = plp.GUROBI_CMD(options=[('TimeLimit', 300)])#
#       milp_prob.solve(solver=solver, file_name=file_name + str(group_counter))
#       result = milp_prob.get_obj_val_dict_old()
#       instance_results.append(result)
#       group_counter += 1
#    instance_results = pd.DataFrame(instance_results)
#    instance_results.to_csv(file_name + '_results.csv')
#    return instance_results, test_instances

# today = datetime.today().strftime('%Y_%m_%d')
# xp_name = f"xp_{today}_split_60_40_Debug"
# #makes a folder of xp_name if it does not exist already

# if not os.path.exists('model_runs/'+ xp_name):
#    os.makedirs('model_runs/'+xp_name)


# start_time = time.time()
# file_name = 'model_runs/'+xp_name+f'/Model_dependent_{NO_STATIONS}_E{NO_EQUIPMENT}_L{NO_WORKERS}_T{SEQUENCE_LENGTH}_C{TAKT_TIME}_LC{WORKER_COST}_'
# results_md_df, instances_md = multi_run(instance_list, model_dependent_problem_linear_labour_recourse, NO_EQUIPMENT,  NO_WORKERS, 
#           NO_STATIONS, WORKER_COST, TAKT_TIME, SEQUENCE_LENGTH, MODEL_MIXTURES, seed, RECOURSE_COST,
#           scenario_generator=make_scenario_tree,
#            file_name=file_name)
# md_time = time.time() - start_time
# start_time = time.time()
# file_name = 'model_runs/'+xp_name+f'/dynamic_{NO_STATIONS}_E{NO_EQUIPMENT}_L{NO_WORKERS}_T{SEQUENCE_LENGTH}_C{TAKT_TIME}_LC{WORKER_COST}_'
# results_md_df, instances_md = multi_run(instance_list, dynamic_problem_linear_labour_recourse, NO_EQUIPMENT,  NO_WORKERS, 
#           NO_STATIONS, WORKER_COST, TAKT_TIME, SEQUENCE_LENGTH, MODEL_MIXTURES, seed, RECOURSE_COST,
#           scenario_generator=make_scenario_tree,
#            file_name=file_name)
# dynamic_time = time.time() - start_time
# print("--- %s seconds ---" % (dynamic_time), "model dependent time", md_time)

In [4]:
# def run_from_config(config_file, milp_model, seed = None, scenario_generator= make_scenario_tree, file_name = 'test', **kwargs):
#    instance_results = []
#    test_instances = []
#    instance_results = []
#    with open(config_file) as f:
#       print('Opening config file', config_file)
#       #Removes file extension from config file name
#       conf_name = config_file.split('.')[0].split('/')[-1]
#       print('conf_name', conf_name)
#       xp_yaml = yaml.load(f, Loader=yaml.FullLoader)
#       SEQUENCE_LENGTH = xp_yaml['sequence_length']
#       NO_WORKERS = xp_yaml['max_workers']
#       NO_STATIONS = xp_yaml['no_stations']
#       WORKER_COST = xp_yaml['worker_cost']
#       RECOURSE_COST = xp_yaml['recourse_cost']
#       group_counter = 0
#       shutil.copyfile(config_file, file_name + conf_name + '_config.yaml')
#       for model_file in xp_yaml['model_files']:
#          test_instance = MultiModelInstance( init_type='model_data_from_yaml',
#                                             model_yaml=model_file, 
#                                             sequence_length=SEQUENCE_LENGTH, 
#                                             max_workers=NO_WORKERS, 
#                                             no_stations=NO_STATIONS, 
#                                             worker_cost=WORKER_COST, 
#                                             recourse_cost=RECOURSE_COST)
#          print('Running instance', test_instance.name)
#          test_instances.append(test_instance)
#          #create equipment
#          if xp_yaml['equipment_files']:
#             print('loading equipment from', xp_yaml['equipment_files'][0])
#             equipment = Equipment(generation_method='import_yaml', equipment_file=xp_yaml['equipment_files'][0])
#             if equipment.no_tasks != test_instance.no_tasks:
#                #raises an error if the equipment and instance have different number of tasks
#                raise ValueError('Equipment and instance have different number of tasks')
#          else:
#             print('creating equipment')
#             NO_EQUIPMENT = xp_yaml['no_equipment']
#             equipment = Equipment(test_instance.no_tasks, NO_EQUIPMENT, NO_STATIONS, generate_equipment, seed)
#          #create scenario tree
#          print('generating scenario tree')
#          model_mixtures = test_instance.model_mixtures
#          scenario_tree_graph, final_sequences = scenario_generator(SEQUENCE_LENGTH, model_mixtures, **kwargs)
#          print('defining problem')
#          milp_prob = milp_model(problem_instance = test_instance, equipment_instance = equipment, sequence_length=SEQUENCE_LENGTH, prod_sequences=final_sequences, **kwargs)
#          #prob = milp_model(test_instance, equipment,test_instance.no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, SEQUENCE_LENGTH, final_sequences, worker_cost=WORKER_COST)
#          start = timer()
#          solver = plp.GUROBI_CMD(options=[('TimeLimit', 300), ('LogFile', file_name+conf_name + str(group_counter) + ".log")])#
#          milp_prob.solve(solver=solver, file_name=file_name + conf_name+ str(group_counter))
#          end = timer()
#          result = milp_prob.get_obj_val_dict()
#          result['run_time'] = end - start
#          instance_results.append(result)
#          group_counter += 1
#          instance_results = pd.DataFrame(instance_results)
#          output_path=file_name + conf_name +  '_results.csv'
#          instance_results.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
     

#    return instance_results, test_instances

# today = datetime.today().strftime('%Y_%m_%d')
# xp_name = f"xp_{today}_config_test2"
# if not os.path.exists('model_runs/'+ xp_name):
#    os.makedirs('model_runs/'+xp_name)
   
# start_time = time.time()
# file_name = 'model_runs/'+xp_name+f'/model_dependent_'
# results_md_df, instances_md = run_from_config('SALBP_benchmark/MM_instances/medium_instance_config_S4.yaml', model_dependent_problem_linear_labor_recourse, 
#           scenario_generator=make_scenario_tree,
#            file_name=file_name)
# md_time = time.time() - start_time
# start_time = time.time()
# file_name = 'model_runs/'+xp_name+f'/dynamic_'
# results_md_df, instances_md =  run_from_config('SALBP_benchmark/MM_instances/medium_instance_config_S4.yaml', dynamic_problem_linear_labor_recourse, 
#           scenario_generator=make_scenario_tree,
#            file_name=file_name)
# dynamic_time = time.time() - start_time
# print("--- %s seconds ---" % (dynamic_time), "model dependent time", md_time)

In [5]:
def run_from_config(config_file, seed = None, scenario_generator= make_scenario_tree, base_file_name = 'test', **kwargs):
   test_instances = []
   with open(config_file) as f:
      print('Opening config file', config_file)
      print('base_file_name', base_file_name)
      #Removes file extension from config file name
      conf_name = config_file.split('.')[0].split('/')[-1]
      print('conf_name', conf_name)
      xp_yaml = yaml.load(f, Loader=yaml.FullLoader)
      #configuring problem
      SEQUENCE_LENGTH = xp_yaml['sequence_length']
      NO_WORKERS = xp_yaml['max_workers']
      NO_STATIONS = xp_yaml['no_stations']
      WORKER_COST = xp_yaml['worker_cost']
      RECOURSE_COST = xp_yaml['recourse_cost']
      #configuring scenario tree generator
      tree_kwargs = {}
      if xp_yaml['scenario_generator']== 'monte_carlo_tree':
         scenario_generator = monte_carlo_tree
         tree_kwargs['n_samples'] = xp_yaml['scenario_generator']['n_samples']
         tree_kwargs['enum_depth'] = xp_yaml['scenario_generator']['enum_depth']
      else:
         scenario_generator = make_scenario_tree
      group_counter = 0
      #copying config file to results folder
      print('copying config file to results folder',  config_file, base_file_name +'/'+ config_file)
      shutil.copyfile(config_file, base_file_name +'/'+ conf_name + '_config.yaml')
      for milp_model in xp_yaml['milp_models']:
         if milp_model == 'model_dependent_problem_multi_labor_recourse':
            milp_model = model_dependent_problem_multi_labor_recourse
            file_name = base_file_name + '/model_dependent/'
            #if model_dependent directory does not exist, make it
            if not os.path.exists(file_name):
               os.makedirs(file_name)
            file_name = file_name + 'md_'
         elif milp_model == 'dynamic_problem_multi_labor_recourse':
            milp_model = dynamic_problem_multi_labor_recourse
            file_name = base_file_name + '/dynamic/'
            #if model_dependent directory does not exist, make it
            if not os.path.exists(file_name):
               os.makedirs(file_name)
            file_name = file_name + 'd_'
         elif milp_model == 'model_dependent_problem_linear_labor_recourse':
            milp_model = model_dependent_problem_linear_labor_recourse
            file_name = base_file_name + '/model_dependent_linear/'
            #if model_dependent directory does not exist, make it
            if not os.path.exists(file_name):
               os.makedirs(file_name)
            file_name = file_name + 'lmd_'
         elif milp_model == 'dynamic_problem_linear_labor_recourse':
            milp_model = dynamic_problem_linear_labor_recourse
            file_name = base_file_name + '/dynamic_linear/'
            #if model_dependent directory does not exist, make it
            if not os.path.exists(file_name):
               os.makedirs(file_name)
            file_name = file_name + 'ld_'
         else:
            raise ValueError('milp_model not recognized')
         start_time = time.time()
         for model_file in xp_yaml['model_files']:
            print('running milp_model', milp_model)
            test_instance = MultiModelTaskTimesInstance( init_type='model_data_from_yaml',
                                             model_yaml=model_file, 
                                             sequence_length=SEQUENCE_LENGTH, 
                                             max_workers=NO_WORKERS, 
                                             no_stations=NO_STATIONS, 
                                             worker_cost=WORKER_COST, 
                                             recourse_cost=RECOURSE_COST)
            print('Running instance', test_instance.name)
            test_instances.append(test_instance)
            #create equipment
            if xp_yaml['equipment_files']:
               print('loading equipment from', xp_yaml['equipment_files'][0])
               equipment = Equipment(generation_method='import_yaml', equipment_file=xp_yaml['equipment_files'][0])
               if equipment.no_tasks != test_instance.no_tasks:
                  print('equipmen no tasks', equipment.no_tasks)
                  print('instance no tasks', test_instance.no_tasks)
                  #raises an error if the equipment and instance have different number of tasks
                  raise ValueError('Equipment and instance have different number of tasks')
            else:
               print('creating equipment')
               NO_EQUIPMENT = xp_yaml['no_equipment']
               equipment = Equipment(test_instance.no_tasks, 
                                     NO_EQUIPMENT, 
                                     NO_STATIONS, 
                                     generate_equipment, 
                                     seed)
            #create scenario tree
            print('generating scenario tree')
            model_mixtures = test_instance.model_mixtures
            scenario_tree_graph, final_sequences = scenario_generator(SEQUENCE_LENGTH, model_mixtures, **tree_kwargs)
            print('defining problem')
            milp_prob = milp_model(problem_instance = test_instance, 
                                   equipment_instance = equipment, 
                                   sequence_length=SEQUENCE_LENGTH, 
                                   prod_sequences=final_sequences)
            start = timer()
            solver = plp.GUROBI_CMD(options=[ ('TimeLimit', 600), ('LogFile', file_name+conf_name + str(group_counter) + ".log")])#
            milp_prob.solve(solver=solver, 
                            file_name=file_name + conf_name+ str(group_counter))
            end = timer()
            result = milp_prob.get_obj_val_dict()
            result['run_time'] = end - start
            result_df = pd.DataFrame(result, index=[0])
            if group_counter == 0:
               results_df = result_df.copy()
            else:
               results_df = pd.concat([results_df, result_df], axis=0, ignore_index=True)
            output_path=file_name + conf_name +  '_results.csv'
            results_df.to_csv(output_path)
            group_counter += 1
         end_time = time.time()
         print('time for', milp_model, end_time - start_time)
   return 1

today = datetime.today().strftime('%Y_%m_%d')
xp_name = f"xp_{today}_mc_debug"
if not os.path.exists('model_runs/'+ xp_name):
   os.makedirs('model_runs/'+xp_name)
   
file_name = 'model_runs/'+xp_name
run_from_config('SALBP_benchmark/MM_instances/small_instance.yaml',
           base_file_name=file_name)





Opening config file SALBP_benchmark/MM_instances/small_instance.yaml
base_file_name model_runs/xp_2023_11_18_mc_debug
conf_name small_instance
copying config file to results folder SALBP_benchmark/MM_instances/small_instance.yaml model_runs/xp_2023_11_18_mc_debug/SALBP_benchmark/MM_instances/small_instance.yaml
running milp_model <class 'milp_models.model_dependent_problem_multi_labor_recourse'>
using this file SALBP_benchmark/MM_instances/model_data/small_instances/2_models/n=20_2_n=20_3.yaml
Running instance n=20_2_n=20_3
loading equipment from SALBP_benchmark/MM_instances/equipment_data/random_O20_E4_S4_seed42.yaml
generating scenario tree
defining problem
Set parameter TimeLimit to value 600
Set parameter LogFile to value "model_runs/xp_2023_11_18_mc_debug/model_dependent/md_small_instance0.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format mode

/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/milp_models.py:1124: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['station', 'model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()
/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/ALB_instance_tools.py:153: UserWarning: Model probabilities do not sum to 1: 0.9999999999999999
  warnings.warn(f'Model probabilities do not sum to 1: {total_probability}')


Set parameter TimeLimit to value 600
Set parameter LogFile to value "model_runs/xp_2023_11_18_mc_debug/model_dependent/md_small_instance1.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/08567cd975ef40fa8fd2873af7b4d334-pulp.lp
Reading time = 0.02 seconds
Total_cost: 5923 rows, 1121 columns, 30096 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5923 rows, 1121 columns and 30096 nonzeros
Model fingerprint: 0x97a0116f
Variable types: 0 continuous, 1121 integer (656 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [8e+00, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 448 rows and 192 columns
Presolve time: 0.10s
Presolved: 547

/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/milp_models.py:1124: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['station', 'model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


running milp_model <class 'milp_models.model_dependent_problem_multi_labor_recourse'>
using this file SALBP_benchmark/MM_instances/model_data/small_instances/2_models/n=20_3_n=20_4.yaml
Running instance n=20_3_n=20_4
loading equipment from SALBP_benchmark/MM_instances/equipment_data/random_O20_E4_S4_seed42.yaml
generating scenario tree
defining problem
Set parameter TimeLimit to value 600
Set parameter LogFile to value "model_runs/xp_2023_11_18_mc_debug/model_dependent/md_small_instance2.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/8889633fdcf146a2ba2979678b999168-pulp.lp
Reading time = 0.02 seconds
Total_cost: 5924 rows, 1121 columns, 30128 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5924 rows, 

...

/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/milp_models.py:1124: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['station', 'model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()


Set parameter TimeLimit to value 600
Set parameter LogFile to value "model_runs/xp_2023_11_18_mc_debug/model_dependent/md_small_instance4.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/84ba43d21ef844f098a2f43b50ecd3b9-pulp.lp
Reading time = 0.02 seconds
Total_cost: 5922 rows, 1121 columns, 30064 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5922 rows, 1121 columns and 30064 nonzeros
Model fingerprint: 0xcc33b31d
Variable types: 0 continuous, 1121 integer (656 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [2e+01, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 448 rows and 193 columns
Presolve time: 0.12s
Presolved: 547

/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/milp_models.py:1124: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  task_assignments_df = task_assignments_df.groupby(['station', 'model'])['task', 'task_times'].agg({'task':lambda x: ','.join(x.astype(str)), 'task_times': sum }).reset_index()
/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/ALB_instance_tools.py:153: UserWarning: Model probabilities do not sum to 1: 0.9999999999999999
  warnings.warn(f'Model probabilities do not sum to 1: {total_probability}')


Set parameter TimeLimit to value 600
Set parameter LogFile to value "model_runs/xp_2023_11_18_mc_debug/model_dependent/md_small_instance5.log"
Using license file /Users/letshopethisworks2/gurobi.lic

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[rosetta2])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/6v/7nrd1rj91hx3tb4q5npbdf0w0000gn/T/abe6081065fa4a7ab6c9138f9d894cb1-pulp.lp
Reading time = 0.03 seconds
Total_cost: 5925 rows, 1121 columns, 30160 nonzeros

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 5925 rows, 1121 columns and 30160 nonzeros
Model fingerprint: 0x3d2380f7
Variable types: 0 continuous, 1121 integer (656 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [5e+01, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 448 rows and 192 columns
Presolve time: 0.11s
Presolved: 547

KeyboardInterrupt: 

In [ ]:

summary = glt.parse("model_runs/xp_2023_10_06_config_test2/dynamic_small_instance_config0.log")
nl = summary.progress("nodelog")
nl.head()

In [ ]:



plot_incumbent_vs_bound("model_runs/xp_2023_10_06_config_test2/", "dynamic_small_instance_config0.log")

In [ ]:
import yaml
NO_EQUIPMENT = 4
seed = 1
NO_WORKERS =4
NO_STATIONS = 4
WORKER_COST = 500
RECOURSE_COST = WORKER_COST * 2
TAKT_TIME = 1000
SEQUENCE_LENGTH = 4
#MODEL_MIXTURES = {'A':0.34, 'B':0.33, 'C':0.33}
MODEL_MIXTURES = {'A':0.60, 'B':0.40}

instances = [
    {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_200.alb', 'name': 'A', 'probability': 0.6},
     #{'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_17.alb', 'name': 'C', 'probability': 0.4},
     {'fp': 'SALBP_benchmark/small data set_n=20/instance_n=20_16.alb', 'name': 'B', 'probability': 0.4}]
#mm_instance = MultiModelInstance( init_type='from_yaml',mm_yaml_file='/Users/letshopethisworks2/Documents/Phd Paper material/MMABPWW/SALBP_benchmark/MM_instances/n=20_1_n=20_2_n=20_3_.yaml', sequence_length=SEQUENCE_LENGTH, max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST, recourse_cost=RECOURSE_COST)



In [ ]:
small_instances = get_instance_list('SALBP_benchmark/MM_instances/model_data', extension='.yaml')

In [ ]:
instance_groups = pair_instances(instances, MODEL_MIXTURES)
mm_instance = MultiModelTaskTimesInstance(model_dicts=instances, takt_time=TAKT_TIME, sequence_length=SEQUENCE_LENGTH, max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST, recourse_cost=RECOURSE_COST)

print(mm_instance.name)
print(mm_instance.data)
print(mm_instance.takt_time)


mm_instance.genererate_task_times(change_func=change_task_times_linear)

mm_instance.model_data_to_yaml(f'SALBP_benchmark/MM_instances/model_data/')


In [ ]:
my_instance = MultiModelTaskTimesInstance(init_type = 'instance_from_yaml',instance_config_yaml='SALBP_benchmark/MM_instances/small_instance_config.yaml', model_yaml= 'SALBP_benchmark/MM_instances/model_data/n=20_200_n=20_25.yaml')

In [ ]:
my_instance.data

In [ ]:
print(mm_instance.data)

In [ ]:
equip = Equipment(mm_instance.no_tasks, NO_EQUIPMENT, NO_STATIONS,  seed, mean = 200, variance = 100)
equip.to_yaml(f'SALBP_benchmark/MM_instances/equipment_data/')

In [ ]:
equip2 = Equipment(mm_instance.all_tasks, NO_EQUIPMENT, NO_STATIONS, generation_method='import_yaml', equipment_file='SALBP_benchmark/MM_instances/equipment_data/random_E4_S4_seed42.yaml')

In [ ]:

save_MALBPW_config('small_instance', NO_WORKERS, NO_STATIONS, 
                   WORKER_COST, RECOURSE_COST, SEQUENCE_LENGTH, 
                   'SALBP_benchmark/MM_instances/small_instance_config.yaml', 
                   equipment_files =[ 'SALBP_benchmark/MM_instances/equipment_data/random_O20_E4_S4_seed42.yaml'], instance_files=small_instances )

In [ ]:
equip2.r_oe.shape

In [ ]:
# instance_groups = pair_instances(instance_list, MODEL_MIXTURES)
# group = instance_groups[0]
# test_instance = MultiModelInstance(group, takt_time=TAKT_TIME, sequence_length=SEQUENCE_LENGTH, 
#                                    max_workers=NO_WORKERS, no_stations=NO_STATIONS, worker_cost=WORKER_COST)
# print('Running instance', test_instance.name)

# #create equipment
# print('creating equipment')
# equipment = Equipment(test_instance.all_tasks, NO_EQUIPMENT, NO_STATIONS, generate_equipment, seed)
# #create scenario tree
# print('generating scenario tree')
# scenario_tree_graph, final_sequences = make_scenario_tree(SEQUENCE_LENGTH, MODEL_MIXTURES)
# print('defining problem')
# milp_prob = dynamic_problem_linear_labour(problem_instance = test_instance, 
#                                           equipment_instance = equipment, sequence_length=SEQUENCE_LENGTH, prod_sequences=final_sequences)
# #prob = milp_model(test_instance, equipment,test_instance.no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, SEQUENCE_LENGTH, final_sequences, worker_cost=WORKER_COST)
# solver = plp.GUROBI_CMD(options=[('TimeLimit', 30)])#
# milp_prob.solve(solver=solver, file_name=file_name + str('blarg'))
# result = milp_prob.get_obj_val_dict()

In [ ]:

task_assignment, labor_assignment = milp_prob.generate_report(file_name=file_name + str('blarg'))

In [ ]:
labor_assignment[labor_assignment['scenario'] == '1'].head(40)

## Warm starting dynamic with model dependent solution for faster solutions



In [ ]:
def dynamic_warm_start_model(model_instance_obj, start_solution, equipment_instance,no_tasks, 
                                     NO_WORKERS, NO_STATIONS,takt_time, sequence_length, 
                                     prod_sequences, worker_cost =100, fix_task_assignment = False):
    '''Uses provided task assignments to warm start the dynamic task assignment version of the problem
    For now we assume that the start solution is the model dependent version of the problem'''

    print('Writing problem')
    print('Number of tasks:', no_tasks, 'Number of workers:', NO_WORKERS, '\n','Number of stations:', NO_STATIONS, 'Takt time:', takt_time, 'Sequence length:', sequence_length, 'worker_cost:', worker_cost)
    workers = list(range(0, NO_WORKERS+1))
    stations = list(range(NO_STATIONS))
    model_instance = model_instance_obj.data
    c_se = equipment_instance.c_se
    R_oe = equipment_instance.r_oe
    equipment = list( range(R_oe.shape[1]))
    takts = list(range(sequence_length+NO_STATIONS-1))
    u_se = plp.LpVariable.dicts('u_se', (stations, equipment), lowBound=0, cat='Binary')
    b_wtsl = plp.LpVariable.dicts('b_wtsl', (prod_sequences.keys(), takts, stations, workers), lowBound=0, cat='Binary') 
    #TODO: maybe make this dictionary work different number of tasks for each model
    x_wsoj = plp.LpVariable.dicts('x_wsoj', (prod_sequences.keys(), stations, range(no_tasks), range(sequence_length) ), lowBound=0, cat='Binary')
    Y_w = plp.LpVariable.dicts('Y_w', (prod_sequences.keys()), lowBound=0, cat='Integer')
    #Adding in the start solution
    print('adding in start solution')
    for v in start_solution.variables():
        if 'x_soi' in v.name:
            model_md = v.name.split('_')[4]
            #change the task number to match with the instances
            o = int(v.name.split('_')[3])
            s = int(v.name.split('_')[2])
            for w in prod_sequences.keys():
                #setting x_wsoj values from the start solution
                for j, model in enumerate(prod_sequences[w]['sequence']):
                    if model_md == model:
                        x_wsoj[w][s][o][j].setInitialValue(round(v.varValue))
        elif 'b_wtsl' in v.name:
            w = int(v.name.split('_')[2])
            t = int(v.name.split('_')[3])
            s = int(v.name.split('_')[4])
            l = int(v.name.split('_')[5])
            b_wtsl[w][t][s][l].setInitialValue(round(v.varValue)) 
        elif 'Y_w' in v.name:
            w = int(v.name.split('_')[2])
            Y_w[w].setInitialValue(round(v.varValue))
        elif 'u_se' in v.name:
            print('equipment', v.name.split('_'))
            s = int(v.name.split('_')[2])
            e = int(v.name.split('_')[3])
            u_se[s][e].setInitialValue(round(v.varValue))
    #Defining LP problem
    prob = plp.LpProblem("stochastic_problem", plp.LpMinimize)
    #Objective function
    prob += (plp.lpSum([c_se[s][e]*u_se[s][e]
                         for s in stations
                           for e in equipment]
                      +
                      [prod_sequences[w]['probability']*Y_w[w]* worker_cost
                         for w in prod_sequences.keys()
                        ]),
                "Total cost")
    #Constraints
    #Constraint 1 -- Must hire Y workers if we use Y workers in a given takt
    for w in prod_sequences.keys():
        for t in takts:
            prob += (plp.lpSum([l*b_wtsl[w][t][s][l] for s in stations for l in workers]) <= Y_w[w], f'Y_w_{w}_{t}')
    #Constraint 2 -- can only assign l number of workers to a station for a given scenario and stage
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                prob += (plp.lpSum([b_wtsl[w][t][s][l] for l in workers]) == 1, f'b_wtsl_{w}_{t}_{s}')
        #Constraint 3 all tasks must be assigned to a station
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            #Not strictly necessary if all models have the same number of tasks, could have just looped over no tasks
            #but this is more general
            for o in range(model_instance[model]['num_tasks']): 
                prob += (plp.lpSum([x_wsoj[w][s][o][j] for s in stations]) == 1, f'x_wsoj_{w}_s_{o}_{j}')
        #Constraint 4 -- sum of task times for assigned tasks must be less than takt time times the number of workers for a given station
    for w in prod_sequences.keys():
        for t in takts:
            for s in stations:
                #Get the model at the current scenario, stage, and station
                if 0<= t-s < sequence_length:
                    j = t-s
                    model = prod_sequences[w]['sequence'][j]
                    task_times = model_instance[model]['task_times']
                    prob += (plp.lpSum([task_times[o]*x_wsoj[w][s][int(o)-1][j] 
                                        for o in task_times]) 
                                        <= 
                                        takt_time*plp.lpSum([l * b_wtsl[w][t][s][l] for l in workers]), f'task_time_wts_{w}_{t}_{s}')

    #Constraint 5 -- tasks can only be assigned to a station with the correct equipment
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for s in stations:
                for o in range(model_instance[model]['num_tasks']):
                    prob += x_wsoj[w][s][o][j] <= plp.lpSum([R_oe[o][e]*u_se[s][e] for  e in equipment]), f'equipment_wsoj_{w}_{s}_{o}_{j}'
        #Constraint 6 -- precedence constraints
    for w in prod_sequences.keys():
        for j, model in enumerate(prod_sequences[w]['sequence']):
            for (pred, suc) in model_instance[model]['precedence_relations']:
                prob += (plp.lpSum([ (s+1)  * x_wsoj[w][s][int(pred)-1][j] for s in stations])
                         <=  
                         plp.lpSum([ (s+1)  * x_wsoj[w][s][int(suc)-1][j] for s in stations]), 
                         f'task{pred} before task{suc} for model{model}, item {j} seq {w}' )
        #Constraint 7 -- non-anticipativity constraints
    for w in prod_sequences.keys():
        for w_prime in prod_sequences.keys():
            if w_prime > w:
                add_non_anticipation(prob, w, w_prime , prod_sequences, model_instance, x_wsoj, sequence_length, NO_STATIONS)

                
    return prob

In [ ]:
def warm_start_dynamic(instance_list, NO_EQUIPMENT,  NO_WORKERS, NO_STATIONS, WORKER_COST, TAKT_TIME, NO_TAKTS, MODEL_MIXTURES, seed, scenario_generator= make_scenario_tree, file_name = 'test', **kwargs):
    instance_groups = pair_instances(instance_list, MODEL_MIXTURES)
    print(instance_groups)
    test_results = []
    group_counter = 0
    test_instances = []
    instance_results = []
    for group in instance_groups:
        test_instance = MultiModelInstance(group)
        print('Running instances', group)
        print('\n test_instance', test_instance)
        test_instances.append(test_instance)
        #create equipment
        print('creating equipment')
        equipment = Equipment(test_instance.all_tasks, NO_EQUIPMENT, NO_STATIONS, generate_equipment, seed)
        #create scenario tree
        print('generating scenario tree')
        scenario_tree_graph, final_sequences = scenario_generator(NO_TAKTS, MODEL_MIXTURES, **kwargs)
        print('defining problem')
        md_prob = model_dependent_eq_linear_labour(test_instance, equipment,test_instance.no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, NO_TAKTS, final_sequences, worker_cost=WORKER_COST)
        print('solving problem')
        solver = plp.GUROBI_CMD(options=[('TimeLimit', 300),( 'LPWarmStart',2)])#
        md_prob.solve(solver=solver)
        print('\n writing results')
        print('model dependent objective value: ', md_prob.objective.value())
        print('\n solving dynamic problem')
        dynamic_prob = dynamic_warm_start_model(test_instance, md_prob, equipment,test_instance.no_tasks, NO_WORKERS, NO_STATIONS, TAKT_TIME, NO_TAKTS, final_sequences, worker_cost=WORKER_COST)
        dynamic_prob.solve(solver=solver)
        md_result = obj_val_dict(group, md_prob.objective.value(),md_prob.status, test_instance.data)
        dynamic_result = obj_val_dict(group, dynamic_prob.objective.value(),dynamic_prob.status, test_instance.data)
        result = {**md_result, **dynamic_result}
        instance_results.append(result)
        print('test_instance', test_instance)
        out_name = file_name + str(group_counter)
        task_assignment, labor_assignment = generate_report_md(md_prob, final_sequences, test_instance.data, out_name + 'md')
        task_assignment, labor_assignment = generate_report_dynamic(dynamic_prob, final_sequences, test_instance.data, out_name + 'dynamic')
        group_counter += 1

    instance_results = pd.DataFrame(instance_results)
    instance_results.to_csv(file_name + '_results.csv')
    return instance_results, test_instances

start_time = time.time()
file_name = 'model_runs/'+xp_name+f'/Warm_start_{NO_STATIONS}_E{NO_EQUIPMENT}_L{NO_WORKERS}_T{NO_TAKTS}_C{TAKT_TIME}_LC{WORKER_COST}_'
print('writing to ', file_name)
results_df, instances = warm_start_dynamic(instance_list, NO_EQUIPMENT,  NO_WORKERS, NO_STATIONS, WORKER_COST, TAKT_TIME, NO_TAKTS, MODEL_MIXTURES, seed, file_name = file_name)
warm_start_time= time.time() - start_time
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
print('time for normal run', normal_time)
print('time for warm start run', warm_start_time)

# Heuristic section

## Constructive heuristic for model dependent task assignment

In [ ]:
#Tasks selection methods
def longest_processing_time(model, candidate_list, **kwargs):
    max_task_time = 0
    for candidate in candidate_list:
        if model['task_times'][candidate] > max_task_time:
            max_task_time = model['task_times'][candidate]
            selected_task = candidate
    return selected_task


#Methods for the construction heurisitc

def calculate_takt_time(model, NO_S, TAKT_TIME):
    """
    Calculate the takt time for the model dependent model
    """
    total_task_time = sum(model['task_times'].values())
    new_takt_time = max(total_task_time/NO_S, TAKT_TIME)
    return new_takt_time

def model_task_assignment(model, all_tasks, NO_S, TAKT_TIME, selection_method, **kwargs):
    '''This function assigns the tasks of a model '''
    print('model', model)
    x_so = np.zeros((NO_S,len(all_tasks)))
    new_takt_time = calculate_takt_time( model, NO_S, TAKT_TIME)
    prec_matrix = construct_precedence_matrix(model)
    number_of_predecessor = np.sum(prec_matrix, axis=0)
    for station in range(NO_S):
        s_total_assingments = 0
        while s_total_assingments < new_takt_time and np.any(number_of_predecessor != -1):
            candidate_list = []
            for task in model['task_times']:
                task_in = int(task)-1
                if number_of_predecessor[task_in] == 0:
                    candidate_list.append(task)
            selected_task = selection_method(model, candidate_list,  **kwargs)
            selected_task_in = int(selected_task)-1
            x_so[station, selected_task_in] = 1
            s_total_assingments += model['task_times'][selected_task]
            number_of_predecessor -= prec_matrix[selected_task_in]
            number_of_predecessor[selected_task_in] = -1
        #If all the elements in number of predecessor are -1, then all tasks are assigned
        if np.all(number_of_predecessor == -1):
            break
    return x_so

def constructive_heurisitc(instance, NO_S, TAKT_TIME, selection_method):
    """
    Constructive heuristic for the model dependent model
    """
    print('instance', instance)
    all_tasks = get_task_union(instance, 'A', 'B')
    print('all_tasks', all_tasks)
    assignments = []
    #heuristic asssigns tasks for each model in instance
    for model in instance:
        x_so = model_task_assignment( instance[model], all_tasks, NO_S, TAKT_TIME, selection_method)
        assignments.append(x_so)
    #combines assignments list into a single numpy array, where eac
    x_soi = np.stack(assignments, axis=-1)
    return x_soi



In [ ]:
def run_constructive_heuristic(instance_list, NO_S, TAKT_TIME, MODEL_MIXTURES):
    """
    Runs the constructive heuristic for all instances in the instance list
    """
    heuristics = []
    instance_groups = pair_instances(instance_list, MODEL_MIXTURES)
    for group in instance_groups:
        instance = create_instance_pair_stochastic(group)
        heuristics.append(constructive_heurisitc(instance, NO_S, TAKT_TIME, longest_processing_time))
    return heuristics

def solve_from_initial_task_asssingments(instance, NO_S, TAKT_TIME, task_assignments):
    """
    Solves the instance using the given task assignments
    """

instance_list = [ "SALBP_benchmark/small data set_n=20/instance_n=20_1.alb",
   "SALBP_benchmark/small data set_n=20/instance_n=20_2.alb",
   ]
NO_S = 4
MODEL_MIXTURES = {'A': 0.5, 'B': 0.5}
results_heuristic = run_constructive_heuristic(instance_list, NO_S, TAKT_TIME, MODEL_MIXTURES)


## Fix and Optimize LNS, starting with model dependent and going to dynamic

Here we will take a solution of the model depedent problem and use it as the first solution for the dynamic case. Here, instead of reoptimizing the entire thing, we choose 2-3 adjacent stations and reoptimize

In [ ]:
def fix_and_optimize(instance, seed, no_iterations = 10, scenario_generator= make_scenario_tree, file_name = 'fix_and_optimize test', **kwargs):
    """
    Fixes the task assignments and solves the instance
    """
    